In [ ]:
import pandas as pd
from konlpy.tag import Okt, Komoran
from soynlp.utils import DoublespaceLineCorpus
from soynlp.vectorizer import sent_to_word_contexts_matrix
from glove import Glove
import re

In [ ]:
!pip install soynlp
!pip install konlpy
!pip install glove_python

In [ ]:
# 리뷰 데이터셋
reviews = pd.read_csv('drive/My Drive/datasets/review_final.csv')
content = reviews['댓글내용']
star = reviews['별점']


# 형태소 분석기
komoran = Komoran() # 상대적으로 많은 데이터를 대상으로 더 빠름, 더 상세히 분석
okt = Okt() # 상대적으로 안정성이 좋음. but 사용자 사전 기능이 없다.

# Komoran 은 실행 중 버그 발생 -> 임시로 Okt 채택


In [ ]:
# tokenizing - DamDam git 출처
def tokenize(doc):
    # 한글 자음, 모음 제거
    doc = re.sub(pattern='([ㄱ-ㅎㅏ-ㅣ]+)', repl='', string=doc)
    # 특수기호 제거
    doc = re.sub(pattern='[^\w\s]', repl='', string=doc)
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    doc = okt.pos(doc, norm=True, stem=True)
    # 명사, 형용사, 부사, 동사 채택
    token = []
    for i in doc:
        if i[1] == 'Noun' or i[1] == 'Verb' or i[1] == 'Adverb' or i[1] == 'Adjective':
          token.append(i)

    return ['/'.join(t) for t in token]

In [ ]:
# Vectorize to co-occurence matrix
x, idx2vocab = sent_to_word_contexts_matrix(
    content,
    windows=3,
    min_tf=10,
    tokenizer=tokenize,
    dynamic_weight=True,
    verbose=True
)

# glove (Word Embedding)
glove = Glove(no_components=100, learning_rate=0.05, max_count=30)
# input coo matrix 변환
glove.fit(x.tocoo(), epochs=5, no_threads=4, verbose=True)

# 사전 추가
dictionary = {vocab:idx for idx, vocab in enumerate(idx2vocab)}
glove.add_dictionary(dictionary)

Create (word, contexts) matrix
  - counting word frequency from 87410 sents, mem=0.909 Gb
  - scanning (word, context) pairs from 87410 sents, mem=1.043 Gb
  - (word, context) matrix was constructed. shape = (6006, 6006)                    
  - done
Performing 5 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4


In [ ]:
def tokened(sentence):
  return ['/'.join(x) for x in okt.pos(sentence)][0]

def get_most_similar(word):
  print(tokened(word))
  print(glove.most_similar(tokened(word), number=10))

In [ ]:
# 유사 단어 테스트
glove.most_similar(tokened('볶음'), number=10)

[('오뎅/Noun', 0.8337399648625302),
 ('탕/Noun', 0.8073953231965109),
 ('샤브샤브/Noun', 0.7304645581988074),
 ('주꾸미/Noun', 0.7113417844249657),
 ('제육/Noun', 0.7101901277544247),
 ('된장/Noun', 0.7026016259594793),
 ('숙주/Noun', 0.701619828402634),
 ('샌드위치/Noun', 0.7007344901125827),
 ('찌게/Noun', 0.690310771903459)]

In [ ]:
# glove model 저장
glove.save('drive/My Drive/datasets/glove_test.model')